# Enhanced Model Comparison

This notebook provides a comprehensive comparison of all trained models, including:
- **Statistical Summary**: Mean, median, std, min, max predictions
- **Performance Metrics**: CV RMSE and Kaggle leaderboard scores
- **Model Rankings**: Best models by CV RMSE and Kaggle score
- **Correlation Analysis**: How similar model predictions are
- **Visual Comparisons**: Distribution plots, boxplots, and heatmaps

> **Note**: This notebook uses enhanced model name matching to properly link submission files with performance metrics.

---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add project root to path
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

from config_local import local_config as config
from scripts.compare_models import load_all_submissions

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

print("Loading model predictions and performance metrics...")
df_predictions = load_all_submissions(config.SUBMISSIONS_DIR)
df_performance = pd.read_csv(config.MODEL_PERFORMANCE_CSV)
df_performance['timestamp'] = pd.to_datetime(df_performance['timestamp'])

# Get latest entry for each model
latest_performance = df_performance.sort_values('timestamp').groupby('model').last().reset_index()

# Create lookup dictionary
perf_lookup = {}
for _, row in latest_performance.iterrows():
    perf_lookup[row['model'].lower()] = {
        'rmse': row['rmse'],
        'kaggle_score': row['kaggle_score'] if pd.notna(row['kaggle_score']) else None,
        'timestamp': row['timestamp'],
        'notes': row['notes']
    }

# Normalize model names (same function as in quick_model_comparison.py)
def normalize_model_name(submission_name):
    name = submission_name.lower()
    if '_' in name:
        parts = name.split('_')
        if len(parts) > 1 and parts[0].isdigit():
            name = '_'.join(parts[1:])
    
    name_mapping = {
        'xgboost': 'xgboost', 'xgb': 'xgboost',
        'lightgbm': 'lightgbm', 'lgb': 'lightgbm', 'lightgb': 'lightgbm',
        'catboost': 'catboost', 'cat': 'catboost',
        'ridge': 'ridge', 'lasso': 'lasso',
        'elasticnet': 'elastic_net', 'elastic_net': 'elastic_net',
        'randomforest': 'random_forest', 'random_forest': 'random_forest', 'rf': 'random_forest',
        'svr': 'svr', 'blending': 'blending', 'blend': 'blending',
        'stacking': 'STACKING_META', 'stack': 'STACKING_META',
        'linearregression': 'linear_regression', 'linear_regression': 'linear_regression',
    }
    
    if name in name_mapping:
        return name_mapping[name]
    for key, value in name_mapping.items():
        if key in name:
            return value
    return name

print(f"\nLoaded {len(df_predictions.columns)} submission files")
print(f"Loaded {len(latest_performance)} model performance entries")

# Filter valid models (exclude exploded predictions)
valid_models = [col for col in df_predictions.columns if df_predictions[col].mean() < 1e7]
invalid_models = [col for col in df_predictions.columns if col not in valid_models]

print(f"Valid models: {len(valid_models)}")
print(f"Invalid models (exploded): {len(invalid_models)}")

if invalid_models:
    print(f"\n⚠️ Invalid models (excluded): {', '.join(invalid_models)}")

# Create comprehensive comparison
comparison_data = []
for model_name in valid_models:
    preds = df_predictions[model_name]
    normalized_name = normalize_model_name(model_name)
    perf_data = perf_lookup.get(normalized_name, {})
    
    comparison_data.append({
        'Model': model_name,
        'Mean ($)': preds.mean(),
        'Median ($)': preds.median(),
        'Std ($)': preds.std(),
        'Min ($)': preds.min(),
        'Max ($)': preds.max(),
        'CV RMSE': perf_data.get('rmse'),
        'Kaggle Score': perf_data.get('kaggle_score'),
        'Notes': perf_data.get('notes', '')
    })

df_comparison = pd.DataFrame(comparison_data)

# Format for display
display_df = df_comparison.copy()
display_df['Mean ($)'] = display_df['Mean ($)'].apply(lambda x: f"${x:,.0f}")
display_df['Median ($)'] = display_df['Median ($)'].apply(lambda x: f"${x:,.0f}")
display_df['Std ($)'] = display_df['Std ($)'].apply(lambda x: f"${x:,.0f}")
display_df['Min ($)'] = display_df['Min ($)'].apply(lambda x: f"${x:,.0f}")
display_df['Max ($)'] = display_df['Max ($)'].apply(lambda x: f"${x:,.0f}")
display_df['CV RMSE'] = display_df['CV RMSE'].apply(lambda x: f"{x:.6f}" if x else "N/A")
display_df['Kaggle Score'] = display_df['Kaggle Score'].apply(lambda x: f"{x:.5f}" if x else "N/A")

print("\n" + "="*80)
print("COMPREHENSIVE MODEL STATISTICS")
print("="*80)
print(display_df.to_string(index=False))

# Ranking by CV RMSE
print("="*80)
print("RANKINGS BY CV RMSE (lower is better)")
print("="*80)

df_ranking_rmse = df_comparison.copy()
df_ranking_rmse = df_ranking_rmse[df_ranking_rmse['CV RMSE'].notna()].sort_values('CV RMSE')

for idx, (_, row) in enumerate(df_ranking_rmse.iterrows(), 1):
    kaggle = f"{row['Kaggle Score']:.5f}" if row['Kaggle Score'] else "N/A"
    print(f"{idx:2d}. {row['Model']:30s} - CV RMSE: {row['CV RMSE']:.6f} | Kaggle: {kaggle}")

# Ranking by Kaggle Score
print("\n" + "="*80)
print("RANKINGS BY KAGGLE SCORE (lower is better)")
print("="*80)

df_ranking_kaggle = df_comparison.copy()
df_ranking_kaggle = df_ranking_kaggle[df_ranking_kaggle['Kaggle Score'].notna()].sort_values('Kaggle Score')

for idx, (_, row) in enumerate(df_ranking_kaggle.iterrows(), 1):
    rmse = f"{row['CV RMSE']:.6f}" if row['CV RMSE'] else "N/A"
    print(f"{idx:2d}. {row['Model']:30s} - Kaggle: {row['Kaggle Score']:.5f} | CV RMSE: {rmse}")

# Winner
if len(df_ranking_kaggle) > 0:
    winner = df_ranking_kaggle.iloc[0]
    print(f"\n🏆 WINNER: {winner['Model']} with Kaggle Score {winner['Kaggle Score']:.5f}")

# Calculate correlation matrix
df_valid = df_predictions[valid_models]
corr_matrix = df_valid.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".3f", cmap="coolwarm", center=1, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title("Correlation Between Model Predictions", fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# Find highly correlated pairs
print("\n" + "="*80)
print("HIGHLY CORRELATED MODEL PAIRS (correlation > 0.95)")
print("="*80)

high_corr_pairs = []
for i, model1 in enumerate(valid_models):
    for model2 in valid_models[i+1:]:
        corr = corr_matrix.loc[model1, model2]
        if corr > 0.95:
            high_corr_pairs.append((model1, model2, corr))

if high_corr_pairs:
    for model1, model2, corr in sorted(high_corr_pairs, key=lambda x: x[2], reverse=True):
        print(f"  {model1:25s} <-> {model2:25s}: {corr:.4f}")
else:
    print("  None found")

## 4. Distribution Comparison

Compare prediction distributions across models and with the actual training target.


In [ ]:
# Load training data for reference (target distribution)
try:
    train_df = pd.read_csv(config.TRAIN_PROCESS8_CSV)
    y_train = np.expm1(train_df["logSP"])
    has_train = True
    print(f"Loaded training data: {len(y_train)} samples")
    print(f"Training target range: ${y_train.min():,.0f} - ${y_train.max():,.0f}")
    print(f"Training target mean: ${y_train.mean():,.0f}")
except Exception as e:
    print(f"Could not load training data: {e}")
    y_train = None
    has_train = False

# Distribution plot (KDE) - Compare predictions vs actual target
plt.figure(figsize=(14, 8))
if has_train:
    sns.kdeplot(y_train, label="TRAIN (Actual Target)", color="black", linewidth=3, linestyle="--", alpha=0.8)

for col in valid_models:
    sns.kdeplot(df_valid[col], label=col, alpha=0.6)

plt.title("SalePrice Distribution: Model Predictions vs. Actual Training Target", fontsize=16, pad=20, fontweight='bold')
plt.xlabel("SalePrice ($)", fontsize=12)
plt.ylabel("Density", fontsize=12)
plt.xlim(0, 800000)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print distribution statistics
if has_train:
    print("\n" + "="*80)
    print("DISTRIBUTION STATISTICS: Predictions vs Target")
    print("="*80)
    print(f"{'Model':<30s} {'Mean':<15s} {'Std':<15s} {'Min':<15s} {'Max':<15s}")
    print("-"*80)
    print(f"{'TRAIN (Actual)':<30s} ${y_train.mean():>13,.0f} ${y_train.std():>13,.0f} ${y_train.min():>13,.0f} ${y_train.max():>13,.0f}")
    for col in valid_models:
        preds = df_valid[col]
        print(f"{col:<30s} ${preds.mean():>13,.0f} ${preds.std():>13,.0f} ${preds.min():>13,.0f} ${preds.max():>13,.0f}")


NameError: name 'plt' is not defined

## 5. Boxplot Comparison

Visualize prediction ranges and outliers compared to the actual training target.


In [ ]:
# Boxplot comparison - Visualize prediction ranges and outliers vs target
df_melted = df_valid.melt(var_name="Model", value_name="SalePrice")

if has_train:
    train_melted = pd.DataFrame({"Model": ["TRAIN (Actual)"] * len(y_train), "SalePrice": y_train})
    df_melted = pd.concat([train_melted, df_melted], ignore_index=True)

plt.figure(figsize=(14, 8))
sns.boxplot(x="SalePrice", y="Model", data=df_melted, orient="h")
plt.title("SalePrice Range Comparison: Model Predictions vs. Actual Training Target", fontsize=16, pad=20, fontweight='bold')
plt.xlabel("SalePrice ($)", fontsize=12)
plt.xlim(0, 800000)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

# Additional histogram comparison: Compare each model's predictions against target distribution
if has_train and len(valid_models) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    # Select top 4 models for detailed comparison
    top_models = valid_models[:4] if len(valid_models) >= 4 else valid_models
    
    for idx, model_name in enumerate(top_models):
        if idx >= 4:
            break
        ax = axes[idx]
        
        # Histogram comparison
        ax.hist(y_train, bins=50, alpha=0.5, label="TRAIN (Actual)", color="black", density=True)
        ax.hist(df_valid[model_name], bins=50, alpha=0.5, label=model_name, density=True)
        
        ax.set_xlabel("SalePrice ($)", fontsize=10)
        ax.set_ylabel("Density", fontsize=10)
        ax.set_title(f"{model_name} vs Target Distribution", fontsize=12, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xlim(0, 800000)
    
    plt.suptitle("Model Predictions vs Training Target Distribution (Histograms)", fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()


## 6. Visual Comparisons (Generated by compare_models.py)

For detailed visualizations including pairwise scatter plots, run:
```bash
python scripts/compare_models.py
```

This generates:
- Correlation heatmap
- Distribution comparison
- Boxplot comparison  
- Pairwise scatter plots for top models

Images are saved to: `runs/latest/comparison/`
